# File systems proxies

This notebook demonstrates file system proxies to folders and directories, which are basically strings (paths) with additional methods for creation, opening moving, deleting, renaming, linking and browsing. Three file systems are currently supported:
 * Local file system
 * HDF5 file system
 * Nexus file system

In [1]:
from spectrocrunch.io import fs,localfs,h5fs,nxfs

This notebook will create directories and files in a context manager that cleans up everything afterwards:

In [2]:
import os
from contextlib import contextmanager
@contextmanager
def temppath(hdf5=False):
    with localfs.Path('.').temp() as root:
        root.mkdir()
        if hdf5:
            root = h5fs.Path(root['test.h5']).mkdir()
            print('HDF5 root: {}'.format(root))
        else:
            print('Local root: {}'.format(root))
        yield root

This context manager will be used as follows:

In [3]:
for hdf5 in [True,False]:
    with temppath(hdf5=hdf5) as root:
        root.ls()
    root.ls()
    print('')

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/HAipMX/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/HAipMX/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/HAipMX/test.h5:/ (does not exist)

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ms8dng
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ms8dng
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Ms8dng (does not exist)



## Proxy

A proxy to an existing or non-existing path (file or directory) can be created from a string:

In [4]:
with temppath() as root:
    path = os.path.join(root.path,'subdir')
    print('String: '+path)
    proxy = localfs.Path(path)
    print('Local proxy: '+str(proxy))
    assert(not proxy.exists)
    print('')
    
    path = os.path.join(root.path,'test.h5:/subdir')
    print('String: '+path)
    proxy = h5fs.Path(path)
    print('HDF5 proxy: '+str(proxy))
    assert(not proxy.exists)

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/sGvJEJ
String: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/sGvJEJ/subdir
Local proxy: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/sGvJEJ/subdir

String: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/sGvJEJ/test.h5:/subdir
HDF5 proxy: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/sGvJEJ/test.h5:/subdir


### Create directories

You can use the proxy's "mkdir" method to create the directory

In [5]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        subdir = root['dir']['subdir']
        print('Device: {}'.format(subdir.device))
        print('Path: '+subdir.path)
        print('Location: '+subdir.location)
        assert(not subdir.exists)
        subdir.mkdir(recursive=True)
        assert(subdir.exists)
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/nDKToD
Device: 
Path: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/nDKToD/dir/subdir
Location: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/nDKToD/dir/subdir

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/F0WAsa/test.h5:/
Device: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/F0WAsa/test.h5
Path: /dir/subdir
Location: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/F0WAsa/test.h5:/dir/subdir



### Browse directories

Browse directories that not necessarily exist using indexing or iterators:

In [6]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        # Indexing
        path = root['dir']['subdir']
        assert(path['.'] == path)
        assert(path['..'] == root['dir'])
        assert(path['..'] == path.parent)
        assert(path == path['..']['subdir'])

        # Iterator
        for p in path.iterup:
            print p
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CrEdFh
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CrEdFh/dir/subdir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CrEdFh/dir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/CrEdFh
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev
/mntdirect/_data_id21_inhouse/wout/dev
/mntdirect/_data_id21_inhouse/wout
/mntdirect/_data_id21_inhouse
/mntdirect
/

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/30s6wg/test.h5:/
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/30s6wg/test.h5:/dir/subdir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/30s6wg/test.h5:/dir
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/30s6wg/test.h5:/



Browse directories that exist using the "ls" method or iterators:

In [7]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir']['subdir'].mkdir()
        for subdir in ['dira','dirb']:
            path[subdir].mkdir()
        
        # Iterator
        for subdir in path:
            print(subdir)
        
        # ls
        root.ls(recursive=True)
        print('')


Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/LWqoVK
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/LWqoVK/dir/subdir/dirb
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/LWqoVK/dir/subdir/dira
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/LWqoVK
  |
  +-dir
    |
    +-subdir
      |
      +-dira
      |
      +-dirb

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/xXMd8T/test.h5:/
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/xXMd8T/test.h5:/dir/subdir/dira
/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/xXMd8T/test.h5:/dir/subdir/dirb
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/xXMd8T/test.h5:/
  |
  +-dir
    |
    +-subdir
      |
      +-dira
      |
      +-dirb



### Create files

In [8]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        if hdf5:
            data = range(10)
        else:
            data = 'Hello World'
        path = root['file1.txt'].mkfile(data=data)
        root.ls(stats=True)
        
        with path.open(mode='r') as fileio:
            print('IO object: {}'.format(type(fileio)))
        content = path.read()
        print('Content: {}'.format(content))
        if hdf5:
            assert(tuple(data)==tuple(content))
        else:
            assert(data==content)
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/gUUDr1
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/gUUDr1
  | @size = 4096
  | @blocks = 0
  | @ctime = 2018-12-04 18:57:39.872151
  | @rdev = 0
  | @dev = 162
  | @nlink = 2
  | @gid = 1621
  | @blksize = 1048576
  | @mtime = 2018-12-04 18:57:39.872151
  | @uid = 80528
  | @atime = 2018-12-04 18:57:39.872535
  | @ino = 360438216
  | @mode = 16893
  |
  +-file1.txt
      @size = 11
      @blocks = 0
      @ctime = 2018-12-04 18:57:39.872634
      @rdev = 0
      @dev = 162
      @nlink = 1
      @gid = 1621
      @blksize = 1048576
      @mtime = 2018-12-04 18:57:39.872657
      @uid = 80528
      @atime = 2018-12-04 18:57:39.872151
      @ino = 360438211
      @mode = 33204
IO object: <type 'file'>
Content: Hello World

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Tqo7PU/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Tqo7PU/te

### Move/rename

Moving files or directories can be done with the "move" method (aliases: "mv" and "rename"). The destination can be a path proxy or a string.

In [9]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir1'].mkdir()['test1.txt'].mkfile(data='Hello World')
        
        path = path.move('test2.txt')
        root.ls(recursive=True)
        
        path = path.move('../dir2/test3.txt')
        root.ls(recursive=True)
        
        path = path.move(root['dir1']['test1.txt'])
        root.ls(recursive=True)
        
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PGDDvt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PGDDvt
  |
  +-dir1
    |
    +-test2.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PGDDvt
  |
  +-dir1
  |
  +-dir2
    |
    +-test3.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/PGDDvt
  |
  +-dir1
  | |
  | +-test1.txt
  |
  +-dir2

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/71BIqA/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/71BIqA/test.h5:/
  |
  +-dir1
    |
    +-test2.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/71BIqA/test.h5:/
  |
  +-dir1
  |
  +-dir2
    |
    +-test3.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/71BIqA/test.h5:/
  |
  +-dir1
  | |
  | +-test1.txt
  |
  +-dir2



### Copy

Moving files or directories can be done with the "copy" method (alias: "cp"). The destination can be a path proxy or a string. Existing destinations are replaced when "force=True".

In [10]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir1'].mkdir()['test1.txt'].mkfile(data='Hello World')
        
        path = path.copy('test2.txt')
        root.ls(recursive=True)
        
        path = path.copy('../dir2/test3.txt')
        root.ls(recursive=True)
        
        try:
            path = path.copy(root['dir1']['test1.txt'])
        except fs.AlreadyExists:
            path = path.copy(root['dir1']['test1.txt'],force=True)
        else:
            raise RuntimeError('Copy should not work')
        root.ls(recursive=True)
        
        try:
            path = path.move(root['dir1']['test2.txt'])
        except fs.AlreadyExists:
            path = path.move(root['dir1']['test2.txt'],force=True)
        else:
            raise RuntimeError('Move should not work')
        root.ls(recursive=True)
        
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Uq3ynT
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Uq3ynT
  |
  +-dir1
    |
    +-test1.txt
    |
    +-test2.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Uq3ynT
  |
  +-dir1
  | |
  | +-test1.txt
  | |
  | +-test2.txt
  |
  +-dir2
    |
    +-test3.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Uq3ynT
  |
  +-dir1
  | |
  | +-test1.txt
  | |
  | +-test2.txt
  |
  +-dir2
    |
    +-test3.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Uq3ynT
  |
  +-dir1
  | |
  | +-test2.txt
  |
  +-dir2
    |
    +-test3.txt

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Vs0J8P/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Vs0J8P/test.h5:/
  |
  +-dir1
    |
    +-test1.txt
    |
    +-test2.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/Vs0J8P/

### Links

Hard and soft links are supported. The difference is that when deleting the link destination, the hard link still exists (file is not actually deleted) while the soft link is broken.

In [11]:
for hdf5 in [False,True]:
    with temppath(hdf5=hdf5) as root:
        path = root['dir1'].mkdir()['file.txt'].mkfile(data='Hello World')
        soft = root['soft.txt'].link(path)
        hard = root['hard.txt'].link(path,soft=False)
        root.ls(recursive=True,stats=hdf5)
        
        soft.linkdest().remove(recursive=True)
        assert(soft.lexists)
        assert(not soft.exists)
        assert(hard.lexists)
        assert(hard.exists)
        root.ls(recursive=True,stats=hdf5)
        
        print('')

Local root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/bsJblg
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/bsJblg
  |
  +-dir1
  | |
  | +-file.txt
  |
  +-hard.txt
  |
  +-soft.txt ---> dir1/file.txt
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/bsJblg
  |
  +-dir1
  |
  +-hard.txt
  |
  +-soft.txt -/-> dir1/file.txt

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/QvzRMw/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/QvzRMw/test.h5:/
  |
  +-dir1
  | |
  | +-file.txt = Hello World
  |
  +-hard.txt = Hello World
  |
  +-soft.txt ---> dir1/file.txt = Hello World
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/QvzRMw/test.h5:/
  |
  +-dir1
  |
  +-hard.txt = Hello World
  |
  +-soft.txt -/-> dir1/file.txt



### Cross-device links

HDF5 also supports cross-device links (called external links):

In [16]:
with temppath(hdf5=True) as root1:
    root2 = h5fs.Path(root1.device.parent['external.h5']).mkdir()
    path = root1['dir1'].mkdir()['a'].mkfile(data='Hello World')
    
    lnk = root2['alnk'].link(path)
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    path = lnk.linkdest().rename('b')
    
    print('\n\nLink destination renamed:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    lnk = root2['blnk'].link(path)
    root2['alnk'].remove()
    
    print('\n\nLink to renamed destination:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    lnk.linkdest().copy(root2['b'])
    
    print('\n\nCopy destination to local tree:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    lnk.linkdest().move(root2['c'])
    
    print('\n\nMove destination to local tree:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    root2['b'].move(root1['dir1'])
    
    print('\n\nMove from local tree to destination:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)
    
    root2['blnk'].remove(recursive=True)
    
    print('\n\nRemove link recursively:')
    root1.ls(recursive=True,stats=True)
    print('')
    root2.ls(recursive=True,stats=True)

HDF5 root: /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/test.h5:/
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/test.h5:/
  |
  +-dir1
    |
    +-a = Hello World

+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/external.h5:/
  |
  +-alnk ---> /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/test.h5:/dir1/a = Hello World


Link destination renamed:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/test.h5:/
  |
  +-dir1
    |
    +-b = Hello World

+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/external.h5:/
  |
  +-alnk -/-> /mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/test.h5:/dir1/a


Link to renamed destination:
+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/DZJQUA/test.h5:/
  |
  +-dir1
    |
    +-b = Hello World

+-/mntdirect/_data_id21_inhouse/wout/dev/SpectroCrunchDev/fluxnorm/D